In [ ]:
!pip install -q transformers
!pip install -q sentencepiece
!git clone https://github.com/aub-mind/arabert
!pip install -q farasapy
!pip install -q pyarabic


     |████████████████████████████████| 2.3MB 25.5MB/s 
     |████████████████████████████████| 3.3MB 43.9MB/s 
     |████████████████████████████████| 901kB 56.0MB/s 
     |████████████████████████████████| 1.2MB 27.2MB/s 
Cloning into 'arabert'...
remote: Enumerating objects: 530, done.
remote: Counting objects: 100% (316/316), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 530 (delta 167), reused 226 (delta 82), pack-reused 214
Receiving objects: 100% (530/530), 4.86 MiB | 27.50 MiB/s, done.
Resolving deltas: 100% (290/290), done.
     |████████████████████████████████| 112kB 33.2MB/s 


In [ ]:
import os
import re
from arabert.preprocess import ArabertPreprocessor
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from tqdm.notebook import tqdm
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


#Load dataset

In [ ]:
df_test = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/test.xlsx')
df_test_labels = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/offLabel.xlsx')
df_test.head()

,tweets
0,@USER اما انت تقعد طول عمرك لا مبدا ولا راي ث...
1,@USER @USER بتخاف نسوانك يزعلوا ولا ايه 😂 اه ...
2,RT @USER: يا عـسانـى نـبـقى يا عـمري حـبايـب ...
3,RT @USER: باقي البيان وينو ما شفنه يا برهان <...
4,@USER @USER اللهم انت الشافي المعافي اشفيه وج...


#Preprocessing

In [ ]:
text_cleaning_re = "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]|[.#،<>:@,\\-_”“٪ًَ]"
def preprocess(text):
  #remove tashkeel
  p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
  text = re.sub(p_tashkeel,"", str(text))
  # text = text.replace('وو', 'و')
  # text = text.replace('يي', 'ي')
  # text = text.replace('اا', 'ا')
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  #remove longation
  text = re.sub(r'(.)\1+', r'\1\1', str(text)) 
  text = re.sub("[إأآا]", "ا", str(text))
  text = re.sub("ى", "ي", str(text))
  # text = re.sub("ؤ", "ء", text)
  # text = re.sub("ئ", "ء", text)
  text = re.sub("ة", "ه", str(text))

  tokens = []
  for token in text.split():
    # if token not in stop_words:
      tokens.append(token)
  return " ".join(tokens)

In [ ]:
arabert_prep = ArabertPreprocessor(model_name='bert-base-arabert')

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


100%|██████████| 241M/241M [00:21<00:00, 10.6MiB/s]

[2021-06-06 14:55:07,280 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [ ]:
x_test = df_test.tweets
x_test = x_test.apply(lambda x: arabert_prep.preprocess(x))
x_test[:10]

0    [مستخدم] اما أنت تقعد طول عمر ك+ لا مبدا و+ لا...
1    [مستخدم] [مستخدم] ب+ تخاف نسوان ك+ يزعل +وا و+...
2    RT [مستخدم] : يا عسانى نبقى يا عمري حبايب و+ ح...
3    RT [مستخدم] : باقي ال+ بيان و+ ينو ما شفن +ه ي...
4    [مستخدم] [مستخدم] اللهم أنت ال+ شافي ال+ معافي...
5    RT [مستخدم] : آآه يا غيث يا حبيب +نا لو تعرف ح...
6    [مستخدم] يا حب يا دنيا جديد +ة يا احلى يا احلى...
7    RT [مستخدم] : # صباح _ ال+ اربعاء < LF > . < L...
8    عاوز أوجه رسال +ة ل+ ال+ دكاتر +ة و+ +ان +ي ب+...
9      [مستخدم] حبيبي يا هندس +ه و+ أنت طيب يا ابا ♥ ️
Name: tweets, dtype: object

In [ ]:
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)


100%|██████████| 241M/241M [00:40<00:00, 10.6MiB/s]

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [ ]:
classes = ["Hate","NOT_Hate"]
pred = []
for tweet in x_test:
  encoded_input = tokenizer(tweet, return_tensors='tf')
  output = model(encoded_input)
  scores = output[0][0].numpy()
  scores = softmax(scores)
  if((scores[0] > scores[1]) & (scores[0] >scores[2])  ):
    pred.append(classes[0])
  else:
    pred.append(classes[1])
  # y_pred.append(int(scores[0] > scores[1] && scores[0] >scores[2]  ))
  # pred1.append(int(paraphrase_results[0] > 0))
  # pred.append(scores)
  # y_pred = np.argmax(pred, axis=1)
  # pred.append(int(round(paraphrase_results[0] * 100)))
pred[:10]

['Hate',
 'Hate',
 'NOT_Hate',
 'NOT_Hate',
 'NOT_Hate',
 'NOT_Hate',
 'NOT_Hate',
 'NOT_Hate',
 'NOT_Hate',
 'NOT_Hate']

# Confusion matrix

In [ ]:
#Label Encoding to convert labels to integer values # Actual label

y_test=df_test_labels.labels
LE = LabelEncoder()
# Actual label
y_test=LE.fit_transform(y_test)
print(y_test)
# Predicted label
y_pred=1-LE.fit_transform(pred)
print(y_pred)

[1 1 0 ... 0 1 1]
[1 1 0 ... 0 0 1]


In [ ]:
print("Accuracy", accuracy_score(y_test, y_pred))

print("\nConfusion matrix\n",confusion_matrix(y_test,y_pred))
print("\nClassification report\n",classification_report(y_test,y_pred))

Accuracy 0.831

Confusion matrix
 [[1410  188]
 [ 150  252]]

Classification report
               precision    recall  f1-score   support

           0       0.90      0.88      0.89      1598
           1       0.57      0.63      0.60       402

    accuracy                           0.83      2000
   macro avg       0.74      0.75      0.75      2000
weighted avg       0.84      0.83      0.83      2000

